In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from utils import *

# Change name of dataframe
changes = pd.DataFrame()

In [2]:
ticker = 'AAPL'
start_date = '2023-05-29'
end_date = '2023-06-02'

# Get data
data = get_data(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [3]:
# Get returns
data = get_returns(data)

In [4]:
# This fuction will get the list of the 10 prior days
data = get_list_of_days(data).fillna(0)

In [5]:
data

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.005649,0.149785,0.279350,0.168835,0.011230,0.033713,0.115116,0.086944,0.128908
1,0.005649,0.149785,0.279350,0.168835,0.011230,0.033713,0.115116,0.086944,0.128908,0.067224
2,0.149785,0.279350,0.168835,0.011230,0.033713,0.115116,0.086944,0.128908,0.067224,-0.134306
3,0.279350,0.168835,0.011230,0.033713,0.115116,0.086944,0.128908,0.067224,-0.134306,-0.033605
4,0.168835,0.011230,0.033713,0.115116,0.086944,0.128908,0.067224,-0.134306,-0.033605,-0.067232
...,...,...,...,...,...,...,...,...,...,...
1145,0.050095,0.016817,0.012192,0.018376,0.064010,-0.133065,0.049689,-0.027840,0.075189,0.063992
1146,0.016817,0.012192,0.018376,0.064010,-0.133065,0.049689,-0.027840,0.075189,0.063992,0.100110
1147,0.012192,0.018376,0.064010,-0.133065,0.049689,-0.027840,0.075189,0.063992,0.100110,-0.050009
1148,0.018376,0.064010,-0.133065,0.049689,-0.027840,0.075189,0.063992,0.100110,-0.050009,0.034300


In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    changes.plot(figsize=(8, 4))
    plt.ylabel("daily change in returns (%)");

In [ ]:
import arviz as az
import numpy as np
import pymc3 as pm

with pm.Model() as Apple_model:
    change_returns = pm.Data("changes", changes["changes"], dims="Date", export_index_as_coords=True)

    nu = pm.Exponential("nu", 0.1, testval=5.0)
    sigma = pm.Exponential("sigma", 50, testval=0.1)
    s = pm.GaussianRandomWalk("s", sigma=sigma, dims="Date")
    volatility_process = pm.Deterministic("volatility_process", pm.math.exp(-2*s) == 0.5, dims="Date")
    r = pm.StudentT("r", nu=nu, sigma=volatility_process, observed=change_returns, dims="Date")

In [ ]:
with Apple_model:
    trace = pm.sample(2000, init="adapt_diag", return_inferencedata=False)

In [ ]:
with Apple_model:
    az.plot_trace(trace, var_names=["nu", "sigma"]);

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
changes.plot(ax=ax)
ax.plot(changes.index, 1 / np.exp(trace["s", ::5].T), "C3", alpha=0.03)
ax.set(title="volatility_process", xlabel="time", ylabel="volatility")
ax.legend(["Apple", "stochastic volatility process"], loc="upper right");